In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 180 kB 63.4 MB/s 
     |████████████████████████████████| 144 kB 67.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=010c2f1c6d742be23947687f2e2907310e4804558dc419b164e199570a03cc58
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import numpy as np
from PIL import Image, ImageDraw 
import PIL
from matplotlib.pyplot import imshow
import wandb

In [ ]:
import os
#replace key
os.environ["WANDB_API_KEY"] = "b4c712113157d9a744432d4033a69538a23c6af7"

In [ ]:
def get_parents(ip_seed):
    np.random.seed(ip_seed)
    rect_shape = list(np.random.randint(10, 900, size =2))
    ellipse_shape = list(np.random.randint(10, 900, size =2))
    theta = np.random.randint(0,361, size =2)
    return rect_shape, ellipse_shape, theta

In [ ]:
def img_with_theta(x,y, theta):
  rect_shape, ellipse_shape, theta = x, y, theta
  img_size = 1000
  img_centre = img_size / 2
  img_theta = PIL.Image.new(mode="RGBA", size=(img_size, img_size), color = "green")
  rect_canvas = PIL.Image.new("RGBA", (1000, 1000), (0, 0, 0, 0))
  ell_canvas = PIL.Image.new("RGBA", (1000,1000), (0, 0, 0, 0))
  draw = ImageDraw.Draw(img_theta)
  draw_rect = ImageDraw.Draw(rect_canvas)
  draw_ell = ImageDraw.Draw(ell_canvas)
  # print(rect_shape)
  rect_x1 = img_centre-(0.5*(rect_shape[0]))
  rect_y1 = img_centre-(0.5*(rect_shape[1]))

  rect_x2 = img_centre+(0.5*(rect_shape[0]))
  rect_y2 = img_centre+(0.5*(rect_shape[1]))

  ellipse_x1 = img_centre-(0.5*(ellipse_shape[0]))
  ellipse_y1 = img_centre-(0.5*(ellipse_shape[1]))

  ellipse_x2 = img_centre+(0.5*(ellipse_shape[0]))
  ellipse_y2 = img_centre+(0.5*(ellipse_shape[1]))

  draw_rect.rectangle(((rect_x1, rect_y1), (rect_x2, rect_y2)), fill = "black")
  draw_ell.ellipse(((ellipse_x1, ellipse_y1), (ellipse_x2, ellipse_y2)), fill = "white")
  rect_canvas = rect_canvas.rotate(theta[0])
  # img_rectangle_theta = 
  ell_canvas = ell_canvas.rotate(theta[1])
  rect_canvas.alpha_composite(ell_canvas)
  img_theta.alpha_composite(rect_canvas)
  return img_theta

In [ ]:
def cal_loss(img):
    im = img.load()
    black = 0
    white = 0
    for x in range(0,1000):
        for y in range(0,1000):
            if im[x,y]==(0,0,0,255):
                black +=1
            elif im[x,y]==(255,255,255,255):
                white +=1
    try:
      loss = (1-white/black)**2
    except:
      loss = (1-white)**2
    return loss, white, black           

In [ ]:
def update(loss, white, black, img, rect_shape, ellipse_shape, theta):
    prob = np.random.randint(0,2)
    r_theta =  theta[0] + 5
    e_theta = theta[1] + 5
    img_e = img_with_theta(rect_shape, ellipse_shape,[theta[0], e_theta])
    img_r = img_with_theta(rect_shape, ellipse_shape, [r_theta,theta[1]])
    loss_r, white_r, black_r = cal_loss(img_r)
    loss_e, white_e, black_e = cal_loss(img_e)
    if min(loss, loss_r, loss_e) == loss_e or min(loss, loss_r, loss_e) == loss_r:
      if min(loss_r, loss_e) == loss_e:
        if e_theta>=360:
          e_theta = e_theta - 360
        return loss_e, white_e, black_e, img_e, rect_shape, ellipse_shape, [theta[0],e_theta]
      else:
        if r_theta>=360:
          r_theta = r_theta - 360
        return loss_r, white_r, black_r, img_r, rect_shape, ellipse_shape, [r_theta,theta[1]]
    if white > black and prob == 0:
      rect_shape[0]  = rect_shape[0] + (learning_rate * alpha)
      rect_shape[1]  = rect_shape[1] + (learning_rate * alpha)
    elif white > black and prob == 1:
      ellipse_shape[0]  = ellipse_shape[0] - (learning_rate * alpha)
      ellipse_shape[1]  = ellipse_shape[1] - (learning_rate * alpha)
    elif black > white and prob == 0:
      rect_shape[0]  = rect_shape[0] - (learning_rate * alpha)
      rect_shape[1]  = rect_shape[1] - (learning_rate * alpha)
    elif black > white and prob == 1:
      ellipse_shape[0]  = ellipse_shape[0] + (learning_rate * alpha)
      ellipse_shape[1]  = ellipse_shape[1] + (learning_rate * alpha)
    img = img_with_theta(rect_shape, ellipse_shape, theta)
    loss, white, black = cal_loss(img)
    return loss, white, black, img, rect_shape, ellipse_shape, theta     

In [ ]:
import os
wandb.init(project="Reinforcement Learning Final Run")
learning_rate = 0.01
alpha = 100
seed = 3021
rect_shape, ell_shape, theta = get_parents(seed)
img = img_with_theta(rect_shape,ell_shape, theta)
loss, num_white, num_black = cal_loss(img)
n_iterations = 0
threshold = 1e-2
curr_dir = "/content/drive/MyDrive/LENS/RL Images/"+str(seed)+"/"+str(threshold)+"-threshold"
os.makedirs(curr_dir)
while(loss>=threshold):
  wandb.log({"Loss": loss})
  wandb.log({"Theta Rectangle": theta[0]})
  wandb.log({"Theta Ellipse": theta[1]})
  wandb.log({"Image": [wandb.Image(img)]})
  wandb.log({"Rect Length": rect_shape[0]})
  wandb.log({"Rect Width": rect_shape[1]})
  wandb.log({"Ell Length": ell_shape[0]})
  wandb.log({"Ell Width": ell_shape[1]})
  wandb.log({"Black Pixels": num_black})
  wandb.log({"White Pixels": num_white})
  imgname = "/content/drive/MyDrive/LENS/RL Images/" +str(seed)+"/"+str(threshold)+"-threshold/"+str(n_iterations)+".png"
  img.save(imgname)
  loss, num_white, num_black, img, rect_shape, ell_shape, theta = update(loss, num_white, num_black, img, rect_shape, ell_shape, theta)
  n_iterations = n_iterations + 1
print("Policy reached threshold", threshold,"in",n_iterations,"steps. \n Current loss:",loss,"\n Learning Rate:",learning_rate)

wandb: Currently logged in as: kritisinghh21 (use `wandb login --relogin` to force relogin)


Policy reached threshold 0.01 in 136 steps. 
 Current loss: 0.009035697911804587 
 Learning Rate: 0.01
